In [1]:
from pyspark.sql import SparkSession

# SparkSessionの初期化
spark = SparkSession.builder.appName("ColumnsOperation").getOrCreate()

data_a = [
    ("A001", 100),
    ("B001", 200),
    ("C001", 300),
    ("C002", 400),
    ("E001", 500),
]
df_a = spark.createDataFrame(data_a, ["id", "value1"])
df_a.printSchema()
df_a.show()

data_b = [
    ("B001", "aaa"),
    ("C001", "bbb"),
    ("D001", "ccc"),
    ("E002", "ddd"),
]
df_b = spark.createDataFrame(data_b, ["id", "value2"])
df_b.printSchema()
df_b.show()

root
 |-- id: string (nullable = true)
 |-- value1: long (nullable = true)

+----+------+
|  id|value1|
+----+------+
|A001|   100|
|B001|   200|
|C001|   300|
|C002|   400|
|E001|   500|
+----+------+

root
 |-- id: string (nullable = true)
 |-- value2: string (nullable = true)

+----+------+
|  id|value2|
+----+------+
|B001|   aaa|
|C001|   bbb|
|D001|   ccc|
|E002|   ddd|
+----+------+



In [2]:
# 内部結合 (inner join)
df_inner = df_a.join(df_b, on=["id"], how="inner")
df_inner.show()

+----+------+------+
|  id|value1|value2|
+----+------+------+
|B001|   200|   aaa|
|C001|   300|   bbb|
+----+------+------+



In [3]:
# 外部結合 (outer join)
df_outer = df_a.join(df_b, on=["id"], how="outer")
df_outer.show()

+----+------+------+
|  id|value1|value2|
+----+------+------+
|A001|   100|  NULL|
|B001|   200|   aaa|
|C001|   300|   bbb|
|C002|   400|  NULL|
|D001|  NULL|   ccc|
|E001|   500|  NULL|
|E002|  NULL|   ddd|
+----+------+------+



In [4]:
# 左外部結合 (left Outer join)
df_left_outer = df_a.join(df_b, on=["id"], how="left")
df_left_outer.show()

+----+------+------+
|  id|value1|value2|
+----+------+------+
|A001|   100|  NULL|
|B001|   200|   aaa|
|C001|   300|   bbb|
|C002|   400|  NULL|
|E001|   500|  NULL|
+----+------+------+



In [5]:
# 右外部結合 (Right Outer Join)
df_right_outer = df_a.join(df_b, on=["id"], how="right")
df_right_outer.show()

+----+------+------+
|  id|value1|value2|
+----+------+------+
|B001|   200|   aaa|
|C001|   300|   bbb|
|D001|  NULL|   ccc|
|E002|  NULL|   ddd|
+----+------+------+



In [6]:
data_c = [
    ("A001", 1, "num1"),
    ("B001", 200, "num2"),
    ("C001", 300, "num3"),
    ("C001", 4, "num4"),
    ("E001", 500, "num5"),
]
df_c = spark.createDataFrame(data_c, ["id", "value1", "value3"])
df_c.printSchema()
df_c.show()

root
 |-- id: string (nullable = true)
 |-- value1: long (nullable = true)
 |-- value3: string (nullable = true)

+----+------+------+
|  id|value1|value3|
+----+------+------+
|A001|     1|  num1|
|B001|   200|  num2|
|C001|   300|  num3|
|C001|     4|  num4|
|E001|   500|  num5|
+----+------+------+



In [7]:
# 複数条件での結合
df_inner_multi = df_a.join(df_c, on=["id", "value1"], how="inner")
df_inner_multi.show()

+----+------+------+
|  id|value1|value3|
+----+------+------+
|B001|   200|  num2|
|C001|   300|  num3|
|E001|   500|  num5|
+----+------+------+



In [8]:
data_d = [
    ("B001", "aaa"),
    ("C001", "bbb"),
    ("D001", "ccc"),
    ("E002", "ddd"),
]
df_d = spark.createDataFrame(data_d, ["no", "value5"])
df_d.printSchema()
df_d.show()

root
 |-- no: string (nullable = true)
 |-- value5: string (nullable = true)

+----+------+
|  no|value5|
+----+------+
|B001|   aaa|
|C001|   bbb|
|D001|   ccc|
|E002|   ddd|
+----+------+



In [9]:
# 等価演算子(==)を使用して異なる列名を条件に結合する
df_inner_equal = df_a.join(df_d, df_a.id == df_d.no, how="inner")
df_inner_equal.show()

+----+------+----+------+
|  id|value1|  no|value5|
+----+------+----+------+
|B001|   200|B001|   aaa|
|C001|   300|C001|   bbb|
+----+------+----+------+



In [10]:
# 列名を変更して統一する
df_d = df_d.withColumnRenamed("no", "id")
df_d.show()

# 統一した列名を用いて結合する
df_inner_renamed = df_a.join(df_d, on=["id"], how="inner")
df_inner_renamed.show()

+----+------+
|  id|value5|
+----+------+
|B001|   aaa|
|C001|   bbb|
|D001|   ccc|
|E002|   ddd|
+----+------+

+----+------+------+
|  id|value1|value5|
+----+------+------+
|B001|   200|   aaa|
|C001|   300|   bbb|
+----+------+------+



In [11]:
# SparkSessionを終了
spark.stop()